IMPORTS

In [1]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

THESE ARE FILES THAT WOULD PROBABLY BE IMPORTED INTO A VOLUME

In [2]:
demopref_data = pd.read_csv('demopref_data')
U_I_merged_df = pd.read_csv('U_I_merged_df')

In [4]:
print(U_I_merged_df)

         code_module code_presentation  id_student  id_site  date  sum_click  \
0                AAA             2013J       11391   546669    -5         16   
1                AAA             2013J       11391   546669    17         15   
2                AAA             2013J       28400   546669     6          4   
3                AAA             2013J       28400   546669     7          1   
4                AAA             2013J       32885   546669    13          3   
...              ...               ...         ...      ...   ...        ...   
10655275         GGG             2014J      687620   897095   147          3   
10655276         GGG             2014J      688819   897095     1          1   
10655277         GGG             2014J      642694   897110    15          1   
10655278         GGG             2014J      675924   897110    39          1   
10655279         GGG             2014J      687821   897110    93          1   

         activity_type  
0            o

THESE WILL PROBABLY BE AT JUST BEFORE THE LAST LAYER OF RECOMMENDATIONS

In [5]:
# Load the user_item_matrix from file
user_item_matrix = pd.read_pickle('user_item_matrix.pkl')
# Loading the demopref kmeans model
with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

In [6]:
user_item_matrix

id_site,526721,526735,526737,526738,526739,526740,526741,526742,526743,526744,...,1016724,1019025,1042373,1042374,1042375,1042376,1046237,1046812,1046866,1049562
id_student,,,,,,,,,,,,,,,,,,,,,
6516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23698,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2698257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2698535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


THESE ARE FUNCTIONS NECESSARY FOR THE BUILD OF THE RECOMMENDATIONS, SO WILL PROBABLY BE IN THE EARLIER LAYERS

In [7]:

# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Define a function to get the K most similar users
def find_similar_users(user_id, user_similarity, k=10):
    # Get the index of the user
    user_index = user_item_matrix.index.tolist().index(user_id)

    # Get the similarity scores for all users (convert to list to sort it)

    similarity_scores = user_similarity[user_index]

    # Sort the users based on their similarity scores

    sorted_indices = similarity_scores.argsort()[::-1]

    # Get the K most similar users
    similar_users = [list(U_I_merged_df['id_student'].unique())[i] for i in sorted_indices if i != user_index][:k]
    return similar_users


In [8]:

def recommend_courses_UI(user_id, user_similarity, user_item_matrix, U_I_merged_df, k=10):
    # Get the K most similar users
    similar_users = find_similar_users(user_id, user_similarity, k=k)

    # Get the courses taken by similar users
    similar_courses = []
    for user in similar_users:
        user_courses = U_I_merged_df.loc[U_I_merged_df['id_student'] == user]['code_module']
        similar_courses.extend(user_courses)

    # Count the frequency of each course
    course_counts = Counter(similar_courses)
    print(course_counts)

    # Get the courses the user is enrolled in
    userT_courses = U_I_merged_df.loc[U_I_merged_df['id_student'] == user_id]['code_module']
    list_userT_courses = list(userT_courses)
    list_userT_courses = Counter(list_userT_courses)
    # Sort the courses by frequency and get the top K courses
    top_courses = [course for course, count in course_counts.most_common(k) if course not in list_userT_courses]

    return top_courses, list_userT_courses


RECOMMENDATIONS BASED ON USER_ITEM_MATRIX

In [9]:
recommended_courses, list_userT_courses = recommend_courses_UI(205719, user_similarity, user_item_matrix, U_I_merged_df, k=3)
print(f"Courses enrolled in by user 205719: {list_userT_courses}")
print(f"Top recommended courses for user 205719: {recommended_courses}")

Counter({'FFF': 1667, 'AAA': 1328, 'GGG': 741})
Courses enrolled in by user 205719: Counter({'AAA': 117})
Top recommended courses for user 205719: ['FFF', 'GGG']


RECOMMEND COURSES BASED ON VLE PREFERENCES & CLUSTER OF TARGET USER


In [20]:
# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

demopref_data = pd.read_csv('demopref_data')


In [21]:
print(demopref_data)

         code_module_AAA  code_module_BBB  code_module_CCC  code_module_DDD  \
0                    0.0              0.0              1.0              0.0   
1                    0.0              0.0              1.0              0.0   
2                    0.0              0.0              1.0              0.0   
3                    0.0              0.0              1.0              0.0   
4                    0.0              0.0              1.0              0.0   
...                  ...              ...              ...              ...   
1905375              0.0              0.0              0.0              1.0   
1905376              0.0              0.0              0.0              1.0   
1905377              0.0              0.0              0.0              1.0   
1905378              0.0              0.0              0.0              1.0   
1905379              0.0              0.0              0.0              1.0   

         code_module_EEE  code_module_FFF  code_mod

In [27]:
similar_students = find_similar_users(205719, user_similarity, k=15)
print(similar_students.shape)

KeyboardInterrupt: 

In [28]:
print(similar_students.shape)

(754208, 49)


In [23]:
target_student_cluster = demopref_data.loc[demopref_data['id_student'] == 205719, 'cluster_label'].values[0]
print(target_student_cluster)

3


In [31]:
# filter the similar students based on their cluster label
similar_students = demopref_data.loc[(demopref_data['cluster_label'] == target_student_cluster & demopref_data['id_student'].isin(similar_students))]
print(similar_students)

Empty DataFrame
Columns: [code_module_AAA, code_module_BBB, code_module_CCC, code_module_DDD, code_module_EEE, code_module_FFF, code_module_GGG, code_presentation_2013B, code_presentation_2013J, code_presentation_2014B, code_presentation_2014J, gender_F, gender_M, region_East Anglian Region, region_East Midlands Region, region_Ireland, region_London Region, region_North Region, region_North Western Region, region_Scotland, region_South East Region, region_South Region, region_South West Region, region_Wales, region_West Midlands Region, region_Yorkshire Region, highest_education_A Level or Equivalent, highest_education_HE Qualification, highest_education_Lower Than A Level, highest_education_No Formal quals, highest_education_Post Graduate Qualification, age_band_0-35, age_band_35-55, age_band_55<=, disability_N, disability_Y, id_student, num_of_prev_attempts, id_assessment, date_submitted, is_banked, score, imd_band, studied_credits, final_result_Distinction, final_result_Fail, final_

In [25]:
# Find the courses that are most frequently taken by the filtered similar students
freq_courses = similar_students.iloc[:, :7].sum().dropna().sort_values(ascending=False)
freq_courses = freq_courses[freq_courses != 0].nlargest(3)
print(freq_courses)

code_module_DDD    473968.0
code_module_CCC    189406.0
code_module_FFF     53649.0
dtype: float64


In [26]:

# Recommend the courses that the target student has not taken yet but are among the most frequently taken courses by the filtered similar students
target_student_courses = demopref_data.loc[demopref_data['id_student'] == 205719].iloc[0, :7]
recommended_courses = freq_courses[~target_student_courses.astype(bool)].index.tolist()
print(f"Recommended courses for student {205719}: {recommended_courses}")


Recommended courses for student 205719: ['code_module_DDD', 'code_module_CCC', 'code_module_FFF']


In [12]:
# Define a function to get the K most similar users
def find_similar_users(user_id, user_similarity, k=10):
    # Get the index of the user
    user_index = user_item_matrix.index.tolist().index(user_id)

    # Get the similarity scores for all users (convert to list to sort it)

    similarity_scores = user_similarity[user_index]

    # Sort the users based on their similarity scores

    sorted_indices = similarity_scores.argsort()[::-1]

    # Get the K most similar users
    similar_users = [list(U_I_merged_df['id_student'].unique())[i] for i in sorted_indices if i != user_index][:k]
    return similar_users




def student_exists(student_id, demopref_data):
    exists = demopref_data['id_student'].isin([student_id]).any()
    return exists 




def recommend_courses(student_id, demopref_data, user_similarity):

    if student_exists(student_id,demopref_data) == False:
        top_courses = demopref_data.iloc[:, :7].sum().dropna().sort_values(ascending=False).nlargest(10)
        print(f"Most Popular Courses : {top_courses}")
        return top_courses
    
    else:
        # select the cluster label of the target student
        target_student_cluster = demopref_data.loc[demopref_data['id_student'] == student_id, 'cluster_label'].values[0]

        # find the similar students based on VLE preferences
        similar_students = find_similar_users(student_id, user_similarity, k=15)

        # filter the similar students based on their cluster label
        similar_students = demopref_data.loc[(demopref_data['cluster_label'] == target_student_cluster & demopref_data['id_student'].isin(similar_students))]

        # Find the courses that are most frequently taken by the filtered similar students
        freq_courses = similar_students.iloc[:, :7].sum().dropna().sort_values(ascending=False)
        freq_courses = freq_courses[freq_courses != 0].nlargest(3)

        # Recommend the courses that the target student has not taken yet but are among the most frequently taken courses by the filtered similar students
        target_student_courses = demopref_data.loc[demopref_data['id_student'] == student_id].iloc[0, :7]
        recommended_courses = freq_courses[~target_student_courses.astype(bool)].index.tolist()
        print(f"Recommended courses for student {student_id}: {recommended_courses}")

        return recommended_courses

In [13]:
# Call the recommend_courses function with the student ID, demopref_data, and user_similarity
recommended_courses = recommend_courses(205719, demopref_data, user_similarity)
print(recommended_courses)

Recommended courses for student 205719: []
[]


## THE STEPS BELOW ARE PROBABLY RELEVANT MORE TO NIFI, PRE TREATEMENT AND DATA INGESTION AND PREPARATION AND THINGS LIKE THAT

THESE MIGHT BE STEPS OF PRE TREATEMENT BEFORE LOADING TO THE KMEANS MODEL

In [ ]:
loaded_data = pd.read_csv('df_full_cleaned_predicted_imdband')

demo_features = ['id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'studied_credits', 'disability']
perf_features = ['code_module', 'code_presentation', 'id_assignment', 'score', 'is_banked', 'num_of_prev_attempts', 'final_result']

additional_demo_data = pd.read_csv('studentInfo.csv')

added_features = additional_demo_data[['id_student', 'code_module', 'code_presentation', 'final_result', 'studied_credits']]

added_features = pd.get_dummies(added_features, columns=['code_module', 'code_presentation'], prefix=['code_module', 'code_presentation'])
# Merge the student information data with the original DataFrame
demopref_data = pd.merge(loaded_data, added_features, on=['id_student', 'code_presentation_2013B','code_presentation_2013J','code_presentation_2014B','code_presentation_2014J', 'code_module_AAA','code_module_BBB','code_module_CCC','code_module_DDD','code_module_EEE','code_module_FFF','code_module_GGG' ] )

# Drop the irrelevant columns
demopref_data.drop(['sum_click', 'id_site'], axis=1, inplace=True)
demopref_data = pd.get_dummies(demopref_data, columns=['final_result'], prefix='final_result')
demopref_data['id_student'] = demopref_data['id_student'].astype(int)
demopref_data['id_assessment'] = demopref_data['id_assessment'].astype(int)

THIS WAS THE CODE TO GENERATE THE USER_ITEM_MATRIX

In [ ]:
user_item_matrix = pd.pivot_table(U_I_merged_df, values='sum_click', index='id_student', columns='id_site', aggfunc='sum', fill_value=0)

In [ ]:
# Predict on the data to get the clusters
# Select the features to be used for clustering (demopref_data already has a cluster_label column)
##cluster_features = ['num_of_prev_attempts', 'id_assessment', 'date_submitted', 'is_banked', 'score', 'imd_band', 'studied_credits', 'final_result_Distinction', 'final_result_Fail', 'final_result_Pass', 'final_result_Withdrawn', 'code_module_AAA', 'code_module_BBB', 'code_module_CCC', 'code_module_DDD', 'code_module_EEE', 'code_module_FFF', 'code_module_GGG', 'code_presentation_2013B', 'code_presentation_2013J', 'code_presentation_2014B', 'code_presentation_2014J', 'gender_F', 'gender_M', 'region_East Anglian Region', 'region_East Midlands Region', 'region_Ireland', 'region_London Region', 'region_North Region', 'region_North Western Region', 'region_Scotland', 'region_South East Region', 'region_South Region', 'region_South West Region', 'region_Wales', 'region_West Midlands Region', 'region_Yorkshire Region', 'highest_education_A Level or Equivalent', 'highest_education_HE Qualification', 'highest_education_Lower Than A Level', 'highest_education_No Formal quals', 'highest_education_Post Graduate Qualification', 'age_band_0-35', 'age_band_35-55', 'age_band_55<=', 'disability_N', 'disability_Y']
##
### cluster_data = demopref_data[cluster_features].copy()
##
### Perform clustering
##clusters = kmeans.predict(demopref_data[cluster_features])
##
### Add the cluster labels as a new column to the original dataframe
##demopref_data['cluster_label'] = clusters